In [496]:
import requests
import re
import json
import copy
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

In [497]:
#define all data

r = requests.get('http://api.herostats.io/heroes/all')
r.status_code
r.json
hero_data = r.json()

pprint(hero_data)

hero_data_by_name = {}
hero_data_by_label = {}
hero_data_by_type_attributes_only = {}
hero_type_attribute = ['BaseInt', 'BaseStr', 'BaseAgi']

for hero in hero_data:
    temp_hero = {}
    temp_hero = hero_data[hero]
    temp_hero_name = temp_hero['Name']
    temp_hero.pop(temp_hero_name, None)
    temp_hero.pop('Patch', None)
    hero_data_by_name[temp_hero_name] = temp_hero

pprint(hero_data_by_name)

for entry in hero_data_by_name:
    attr_dict = {}
    for attribute in hero_data_by_name[entry]:
        if attribute in hero_type_attribute:
            attr_dict[attribute] = hero_data_by_name[entry][attribute]
        hero_data_by_type_attributes_only[entry] = attr_dict

pprint(hero_data_by_type_attributes_only)

for entry in hero_data_by_name:
    hero_data_by_label[entry] = find_label(entry, hero_data_by_type_attributes_only)

pprint(hero_data_by_label)

RandomInit = []

while len(RandomInit)<3:
    num = np.random.randint(1,112)
    if num not in RandomInit:
        RandomInit.append(num)

# centroids = {
#     1: hero_data_by_type_attributes_only['Zeus'],
#     2: hero_data_by_type_attributes_only['Troll Warlord'],
#     3: hero_data_by_type_attributes_only['Tiny']
# }
# def random_centroids():
#     centroids = {
#         1: hero_data_by_type_attributes_only.values()[RandomInit[0]],
#         2: hero_data_by_type_attributes_only.values()[RandomInit[1]],
#         3: hero_data_by_type_attributes_only.values()[RandomInit[2]]
#     }
#     return centroids
def random_centroids():
    centroids = {
        1: hero_data_by_type_attributes_only['Sand King'],
        2: hero_data_by_type_attributes_only['Clinkz'],
        3: hero_data_by_type_attributes_only['Slark']
    }
    return centroids

{u'1': {u'AgiGain': 1.4,
        u'Alignment': 0,
        u'Armor': 2.68,
        u'AttackPoint': 0.467,
        u'AttackSwing': 0.863,
        u'BaseAgi': 12,
        u'BaseAttackTime': 1.7,
        u'BaseInt': 16,
        u'BaseStr': 22,
        u'CastPoint': 0.65,
        u'CastSwing': 0.5,
        u'DayVision': 1800,
        u'HP': 598,
        u'HPRegen': 0.25,
        u'ID': 1,
        u'IntGain': 1.8,
        u'Legs': 2,
        u'Mana': 208,
        u'ManaRegen': 0.01,
        u'MaxDmg': 56,
        u'MinDmg': 46,
        u'Movespeed': 310,
        u'Name': u'Earthshaker',
        u'NightVision': 800,
        u'Patch': u'6.86c',
        u'PrimaryStat': 0,
        u'ProjectileSpeed': 0,
        u'Range': 128,
        u'StrGain': 2.9,
        u'Turnrate': 0.9},
 u'10': {u'AgiGain': 1.2,
         u'Alignment': 0,
         u'Armor': 1.54,
         u'AttackPoint': 0.35,
         u'AttackSwing': 0.65,
         u'BaseAgi': 11,
         u'BaseAttackTime': 1.7,
         u'BaseInt': 25,


        u'IntGain': 1.6,
        u'Legs': 2,
        u'Mana': 182,
        u'ManaRegen': 0.01,
        u'MaxDmg': 67,
        u'MinDmg': 61,
        u'Movespeed': 285,
        u'Name': u'Tiny',
        u'NightVision': 800,
        u'Patch': u'6.86c',
        u'PrimaryStat': 0,
        u'ProjectileSpeed': 0,
        u'Range': 128,
        u'StrGain': 3,
        u'Turnrate': 0.5},
 u'30': {u'AgiGain': 2.8,
         u'Alignment': 0,
         u'Armor': 2.08,
         u'AttackPoint': 0.3,
         u'AttackSwing': 0.6,
         u'BaseAgi': 22,
         u'BaseAttackTime': 1.45,
         u'BaseInt': 15,
         u'BaseStr': 22,
         u'CastPoint': 0.3,
         u'CastSwing': 0.47,
         u'DayVision': 1800,
         u'HP': 560,
         u'HPRegen': 0.25,
         u'ID': 30,
         u'IntGain': 1.8,
         u'Legs': 1,
         u'Mana': 195,
         u'ManaRegen': 0.01,
         u'MaxDmg': 53,
         u'MinDmg': 49,
         u'Movespeed': 315,
         u'Name': u'Anti-Mage',
         u'

         u'Turnrate': 0.6},
 u'79': {u'AgiGain': 2,
         u'Alignment': 1,
         u'Armor': 1.1,
         u'AttackPoint': 0.46,
         u'AttackSwing': 0.54,
         u'BaseAgi': 15,
         u'BaseAttackTime': 1.7,
         u'BaseInt': 18,
         u'BaseStr': 18,
         u'CastPoint': 0.4,
         u'CastSwing': 1.1,
         u'DayVision': 1800,
         u'HP': 522,
         u'HPRegen': 0.25,
         u'ID': 79,
         u'IntGain': 3.25,
         u'Legs': 0,
         u'Mana': 234,
         u'ManaRegen': 0.01,
         u'MaxDmg': 51,
         u'MinDmg': 42,
         u'Movespeed': 315,
         u'Name': u'Lich',
         u'NightVision': 800,
         u'Patch': u'6.86c',
         u'PrimaryStat': 2,
         u'ProjectileSpeed': 900,
         u'Range': 550,
         u'StrGain': 1.55,
         u'Turnrate': 0.5},
 u'8': {u'AgiGain': 1.75,
        u'Alignment': 0,
        u'Armor': 5.1,
        u'AttackPoint': 0.433,
        u'AttackSwing': 0.567,
        u'BaseAgi': 15,
        u'Ba

                    u'HP': 617,
                    u'HPRegen': 0.25,
                    u'ID': 50,
                    u'IntGain': 1.5,
                    u'Legs': 2,
                    u'Mana': 195,
                    u'ManaRegen': 0.01,
                    u'MaxDmg': 64,
                    u'MinDmg': 58,
                    u'Movespeed': 300,
                    u'Name': u'Faceless Void',
                    u'NightVision': 800,
                    u'PrimaryStat': 1,
                    u'ProjectileSpeed': 0,
                    u'Range': 128,
                    u'StrGain': 1.8,
                    u'Turnrate': 1},
 u'Gyrocopter': {u'AgiGain': 2.8,
                 u'Alignment': 0,
                 u'Armor': 4.36,
                 u'AttackPoint': 0.2,
                 u'AttackSwing': 0.97,
                 u'BaseAgi': 24,
                 u'BaseAttackTime': 1.7,
                 u'BaseInt': 19,
                 u'BaseStr': 18,
                 u'CastPoint': 0.3,
              

 u'Troll Warlord': {u'AgiGain': 2.75,
                    u'Alignment': 0,
                    u'Armor': 1.94,
                    u'AttackPoint': 0.3,
                    u'AttackSwing': 0.3,
                    u'BaseAgi': 21,
                    u'BaseAttackTime': 1.7,
                    u'BaseInt': 13,
                    u'BaseStr': 17,
                    u'CastPoint': 0.51,
                    u'CastSwing': 0.3,
                    u'DayVision': 1800,
                    u'HP': 503,
                    u'HPRegen': 0.25,
                    u'ID': 97,
                    u'IntGain': 1,
                    u'Legs': 2,
                    u'Mana': 169,
                    u'ManaRegen': 0.01,
                    u'MaxDmg': 56,
                    u'MinDmg': 38,
                    u'Movespeed': 305,
                    u'Name': u'Troll Warlord',
                    u'NightVision': 800,
                    u'PrimaryStat': 1,
                    u'ProjectileSpeed': 1200,
            

In [498]:
# define all functions
def distance(x, y):
    dist = float(0)
    for key in x.keys():
        dist += np.square(x[key]-y[key])
    return np.sqrt(dist)

def find_label(hero_name, hero_dict):
    dist1 = distance(hero_dict[hero_name], centroids[1])
    dist2 = distance(hero_dict[hero_name], centroids[2])
    dist3 = distance(hero_dict[hero_name], centroids[3])
    if (dist1 < dist2) and (dist1 < dist3):
        return 1
    elif (dist2 < dist1) and (dist2 < dist3):
        return 2
    else:
        return 3

def find_centroid(num, hero_dict):
    count = count_per_cluster()
    count = count[num-1]
    attributes = hero_dict['Zeus']
    vals = { attribute:0 for attribute in attributes}
    for entry in hero_data_by_label:
        if hero_data_by_label[entry] == num:
            for attribute in attributes:
                vals[attribute]+=hero_dict[entry][attribute]
    for val in vals:
        vals[val] = vals[val]/float(count)
    return vals

def count_per_cluster():
    count_label_1 = 0
    count_label_2 = 0
    count_label_3 = 0
    for entry in hero_data_by_label:
        if hero_data_by_label[entry] == 1:
            count_label_1+=1
        elif hero_data_by_label[entry] == 2:
            count_label_2+=1
        elif hero_data_by_label[entry] == 3:
            count_label_3+=1
    return count_label_1, count_label_2, count_label_3

def knn(hero_dict, centroids = None):
    old_centroids = copy.deepcopy(centroids)
    while True:
        for hero in hero_data_by_name:
            hero_data_by_label[hero] = find_label(hero, hero_dict)
        for i in range(1,4):
            centroids[i] = find_centroid(i, hero_dict)
        if centroids == old_centroids:
            break
            print "breaking"
        old_centroids = copy.deepcopy(centroids)
        print "not breaking"
    pprint(hero_data_by_label)
    pprint(centroids)
    print count_per_cluster()

In [499]:
knn(hero_data_by_type_attributes_only)
# knn(hero_data_by_type_attributes_only)
# knn(hero_data_by_type_attributes_only)
# knn(hero_data_by_type_attributes_only)
# knn(hero_data_by_type_attributes_only)
# knn(hero_data_by_type_attributes_only)
# knn(hero_data_by_type_attributes_only)

TypeError: 'NoneType' object does not support item assignment

In [ ]:
# name_for_plotting = []
# label_for_plotting = []
# for entry in hero_data_by_label:
# #     name_for_plotting.append(entry)
# #     label_for_plotting.append(hero_data_by_label[entry])

# # print name_for_plotting
# # print label_for_plotting

# # hero_types = ['Int', 'Agi', 'Str']
# # for i in range(len(label_for_plotting)):
# #     label_for_plotting[i] = hero_types[label_for_plotting[i]-1]
    
# # print label_for_plotting

IntHeroes = []
AgiHeroes = []
StrHeroes = []
for entry in hero_data_by_label:
    if hero_data_by_label[entry] == 1:
        IntHeroes.append(entry)
    elif hero_data_by_label[entry] == 2:
        AgiHeroes.append(entry)
    else:
        StrHeroes.append(entry)

print IntHeroes
print AgiHeroes
print StrHeroes

In [ ]:
# string = "Ancient Apparition	Ancient Apparition	295	49.0	522	1.80	578	325	600	18	20	25	INTBane	Bane	310	58.0	598	3.52	724	286	400	22	22	22	INTBatrider	Batrider	290	40.0	598	2.10	673	312	375	22	15	24	INTChen	Chen	300	50.5	560	1.10	596	273	600	23	15	21	INTCrystal Maiden	Crystal Maiden	280	41.0	484	1.24	520	273	600	16	16	16	INTDark Seer	Dark Seer	300	63.0	598	6.68	837	377	128 (Melee)	22	12	25	INTDazzle	Dazzle	305	50.0	484	1.94	540	351	550	16	21	27	INTDeath Prophet	Death Prophet	310	50.0	541	2.96	637	260	600	19	14	23	INTDisruptor	Disruptor	300	51.0	541	1.10	576	286	600	19	15	22	INTEnchantress	Enchantress	335	52.0	484	0.66	503	208	550	16	19	16	INTEnigma	Enigma	300	45.0	503	3.96	622	260	500	17	14	20	INTInvoker	Invoker	280	38.0	541	1.80	599	286	600	17	20	22	INTJakiro	Jakiro	290	57.0	636	2.40	727	364	400	25	10	28	INTKeeper of the Light	Keeper of the Light	315	46.0	446	1.10	475	286	600	14	15	22	INTLeshrac	Leshrac	320	43.0	484	3.22	577	338	600	16	23	26	INTLich	Lich	315	46.5	522	1.10	556	234	550	18	15	18	INTLina	Lina	295	49.0	522	1.24	560	351	670	18	16	27	INTLion	Lion	290	52.0	484	1.10	515	286	600	16	15	22	INTNature's Prophet	Nature's Prophet	295	52.0	541	3.52	655	273	600	19	18	25	INTNecrophos	Necrophos	290	47.0	484	3.10	574	286	550	16	15	22	INTOgre Magi	Ogre Magi	295	61.0	617	7.96	911	221	128 (Melee)	23	14	17	INTOracle	Oracle	305	48.0	522	2.10	587	299	620	18	15	23	INTOutworld Devourer	Outworld Devourer	315	53.5	541	5.36	714	338	450	19	24	26	INTPuck	Puck	295	52.5	465	2.08	523	312	550	15	22	24	INTPugna	Pugna	330	49.0	503	1.24	540	338	600	17	16	26	INTQueen of Pain	Queen of Pain	295	53.0	484	1.52	528	312	550	16	18	24	INTRubick	Rubick	290	49.0	541	0.96	572	351	600	19	14	27	INTShadow Demon	Shadow Demon	295	52.0	503	2.52	579	338	500	17	18	23	INTShadow Shaman	Shadow Shaman	285	50.5	541	1.24	581	273	500	21	16	21	INTSilencer	Silencer	295	50.0	503	1.24	540	351	600	17	16	27	INTSkywrath Mage	Skywrath Mage	325	44.0	541	-0.18	535	351	600	19	13	27	INTStorm Spirit	Storm Spirit	290	50.0	541	5.08	705	299	480	19	22	23	INTTechies	Techies	270	30.0	503	6.96	713	286	700	17	14	22	INTTinker	Tinker	305	52.0	503	3.82	618	351	550	17	13	27	INTVisage	Visage	285	50.0	598	-0.46	581	312	600	22	11	24	INTWarlock	Warlock	295	51.0	522	2.40	597	312	600	22	10	24	INTWindrunner	Windrunner	295	50.0	465	1.38	503	286	600	15	17	22	INTWinter Wyvern	Winter Wyvern	285	41.5	636	1.24	683	325	425	24	16	25	INTWitch Doctor	Witch Doctor	305	56.0	484	0.82	507	312	600	16	13	24	INTZeus	Zeus	295	45.0	541	1.54	590	260	350	19	11	20	INT"
# string = "Anti-Mage	Anti-Mage	315	51.0	560	2.08	629	195	128 (Melee)	22	22	15	AGIArc Warden	Arc Warden	295	45.0	636	0.10	639	312	625	24	15	24	AGIBloodseeker	Bloodseeker	290	56.0	617	3.36	741	234	128 (Melee)	23	24	18	AGIBounty Hunter	Bounty Hunter	315	52.0	503	5.94	682	247	128 (Melee)	17	21	19	AGIBroodmother	Broodmother	295	47.0	503	2.52	579	234	128 (Melee)	17	18	18	AGIClinkz	Clinkz	300	40.0	465	2.08	523	208	630	15	22	16	AGIDrow Ranger	Drow Ranger	300	49.5	503	0.64	522	195	625	17	26	15	AGIEmber Spirit	Ember Spirit	310	54.0	541	1.08	576	260	128 (Melee)	19	22	20	AGIFaceless Void	Faceless Void	300	61.0	617	3.94	762	195	128 (Melee)	23	23	15	AGIGyrocopter	Gyrocopter	315	46.0	522	4.36	658	299	365	18	24	19	AGIJuggernaut	Juggernaut	305	46.0	560	2.80	654	182	128 (Melee)	20	26	14	AGILone Druid	Lone Druid	325	48.0	503	3.36	604	169	550	17	24	13	AGILuna	Luna	330	51.0	465	4.08	578	208	330	15	18	16	AGIMedusa	Medusa	290	47.0	446	1.80	494	247	600	14	20	19	AGIMeepo	Meepo	315	42.0	617	4.22	773	260	128 (Melee)	23	23	20	AGIMirana	Mirana	300	43.5	503	1.80	557	221	630	17	20	17	AGIMorphling	Morphling	285	41.5	541	0.66	562	221	350	19	24	17	AGINaga Siren	Naga Siren	320	45.0	579	5.94	785	234	128 (Melee)	21	21	21	AGINyx Assassin	Nyx Assassin	300	51.0	522	3.66	636	234	128 (Melee)	18	19	18	AGIPhantom Assassin	Phantom Assassin	310	47.0	560	4.22	701	169	128 (Melee)	20	23	15	AGIPhantom Lancer	Phantom Lancer	290	56.0	522	3.22	622	273	128 (Melee)	18	29	18	AGIRazor	Razor	295	46.0	579	2.08	651	247	475	21	22	19	AGIRiki	Riki	290	40.0	503	4.76	646	182	128 (Melee)	17	34	14	AGIShadow Fiend	Shadow Fiend	305	38.0	465	0.80	487	234	500	15	20	18	AGISlark	Slark	305	58.0	579	1.94	646	208	128 (Melee)	21	21	16	AGISniper	Sniper	290	39.0	484	1.94	540	195	550	16	21	15	AGISpectre	Spectre	295	48.0	541	3.22	645	208	128 (Melee)	19	23	16	AGITemplar Assassin	Templar Assassin	305	56.0	522	4.22	654	260	140	18	23	20	AGITerrorblade	Terrorblade	315	51.0	465	7.08	662	247	128 (Melee)	15	22	19	AGITroll Warlord	Troll Warlord	305	47.0	503	1.94	561	169	500	17	21	13	AGIUrsa	Ursa	310	47.0	617	5.52	821	208	128 (Melee)	23	18	16	AGIVengeful Spirit	Vengeful Spirit	300	46.0	484	3.78	593	195	400	18	27	15	AGIVenomancer	Venomancer	285	42.0	522	3.08	618	234	450	18	22	18	AGIViper	Viper	285	45.0	560	1.94	625	195	575	20	21	15	AGIWeaver	Weaver	290	58.0	465	0.96	491	195	425	15	14	15	AGI"
# string = re.sub("\t", '', string)
# string = re.sub("\(Melee\)", '', string)
# string = re.sub("[+-]?\d+(?:\.\d+)*.AGI", '"', string)
# string = re.sub("[+-]?\d+(?:\.\d+)?.", '"\n', string)

# print string
# for i  in range(len(StrHeroes)):
#     StrHeroes[i] = StrHeroes[i][:len(StrHeroes[i])/2]

# print StrHeroes

In [500]:
IntHeroes = [ "Ancient Apparition", "Bane", "Batrider", "Chen", "Crystal Maiden", "Dark Seer", "Dazzle",
    "Death Prophet", "Disruptor", "Enchantress", "Enigma", "Invoker", "Jakiro", "Keeper of the Light", "Leshrac", 
    "Lich", "Lina", "Lion", "Nature's Prophet", "Necrophos", "Ogre Magi", "Oracle", "Outworld Devourer", "Puck",
    "Pugna", "Queen of Pain", "Rubick", "Shadow Demon", "Shadow Shaman", "Silencer",    "Skywrath Mage",
    "Storm Spirit", "Techies", "Tinker", "Visage", "Warlock", "Windrunner", "Winter Wyvern", "Witch Doctor", "Zeus"
]
StrHeroes = ['Abaddon', 'Alchemist', 'Axe', 'Beastmaster', 'Brewmaster', 'Bristleback', 'Centaur Warrunner',
    'Chaos Knight', 'Clockwerk', 'Doom', 'Dragon Knight', 'Earth Spirit', 'Earthshaker', 'Elder Titan', 'Huskar',
     'Io', 'Kunkka', 'Legion Commander', 'Lifestealer', 'Lycan', 'Magnus', 'Night Stalker', 'Omniknight', 'Phoenix',
     'Pudge', 'Sand King', 'Slardar', 'Spirit Breaker', 'Sven', 'Tidehunter', 'Timbersaw', 'Tiny', 'Treant Protector',
     'Tusk', 'Undying', 'Wraith King'
]
AgiHeroes = ['Anti-Mage', 'Arc Warden', 'Bloodseeker', 'Bounty Hunter', 'Broodmother', 'Clinkz', 'Drow Ranger',
    'Ember Spirit', 'Faceless Void', 'Gyrocopter', 'Juggernaut', 'Lone Druid', 'Luna', 'Medusa', 'Meepo', 'Mirana',
    'Morphling', 'Naga Siren', 'Nyx Assassin', 'Phantom Assassin', 'Phantom Lancer', 'Razor', 'Riki', 'Shadow Fiend',
    'Slark', 'Sniper', 'Spectre', 'Templar Assassin', 'Terrorblade', 'Troll Warlord', 'Ursa', 'Vengeful Spirit',
    'Venomancer', 'Viper', 'Weaver'
]

In [501]:
# correct = 0
# correctAgi = 0
# correctStr = 0
# correctInt = 0

# for entry in hero_data_by_label:
#     if hero_data_by_label[entry] == 1:
#         if entry in IntHeroes:
#             correctInt+=1
#             correct+=1
#     elif hero_data_by_label[entry] == 2:
#         if entry in AgiHeroes:
#             correctAgi+=1
#             correct+=1
#     elif hero_data_by_label[entry] == 3:
#         if entry in StrHeroes:
#             correctStr+=1
#             correct+=1
# print correct, correctInt, correctAgi, correctStr
# print count_per_cluster()
def checkcorrectness():
    knn(hero_data_by_type_attributes_only, random_centroids())
    PredictedOne = []
    PredictedTwo = []
    PredictedThree = []
    for entry in hero_data_by_label:
        if hero_data_by_label[entry] == 1:
            PredictedOne.append(entry)
        elif hero_data_by_label[entry] == 2:
            PredictedTwo.append(entry)
        else:
            PredictedThree.append(entry)
    len1 = len(list(set(PredictedOne) & set(IntHeroes)))
    if len1<len(list(set(PredictedTwo) & set(IntHeroes))): len1 = len(list(set(PredictedTwo) & set(IntHeroes))) 
    if len1<len(list(set(PredictedThree) & set(IntHeroes))): len1 = len(list(set(PredictedThree) & set(IntHeroes)))
    len2 = len(list(set(PredictedOne) & set(AgiHeroes)))
    if len2<len(list(set(PredictedTwo) & set(AgiHeroes))): len2 = len(list(set(PredictedTwo) & set(AgiHeroes)))
    if len2<len(list(set(PredictedThree) & set(AgiHeroes))): len2 = len(list(set(PredictedThree) & set(AgiHeroes)))
    len3 = len(list(set(PredictedOne) & set(StrHeroes)))
    if len3<len(list(set(PredictedTwo) & set(StrHeroes))): len3 = len(list(set(PredictedTwo) & set(StrHeroes)))
    if len3<len(list(set(PredictedThree) & set(StrHeroes))): len3 = len(list(set(PredictedThree) & set(StrHeroes)))
    print len1, len2, len3
    
for i in xrange(100):
    checkcorrectness()

not breaking
{u'Abaddon': 2,
 u'Alchemist': 3,
 u'Ancient Apparition': 3,
 u'Anti-Mage': 1,
 u'Arc Warden': 3,
 u'Axe': 2,
 u'Bane': 1,
 u'Batrider': 3,
 u'Beastmaster': 2,
 u'Bloodseeker': 1,
 u'Bounty Hunter': 1,
 u'Brewmaster': 1,
 u'Bristleback': 2,
 u'Broodmother': 1,
 u'Centaur Warrunner': 2,
 u'Chaos Knight': 2,
 u'Chen': 2,
 u'Clinkz': 1,
 u'Clockwerk': 2,
 u'Crystal Maiden': 2,
 u'Dark Seer': 3,
 u'Dazzle': 3,
 u'Death Prophet': 3,
 u'Disruptor': 3,
 u'Doom': 2,
 u'Dragon Knight': 1,
 u'Drow Ranger': 1,
 u'Earth Spirit': 2,
 u'Earthshaker': 2,
 u'Elder Titan': 3,
 u'Ember Spirit': 1,
 u'Enchantress': 1,
 u'Enigma': 3,
 u'Faceless Void': 1,
 u'Gyrocopter': 1,
 u'Huskar': 2,
 u'Invoker': 3,
 u'Io': 3,
 u'Jakiro': 3,
 u'Juggernaut': 1,
 u'Keeper of the Light': 3,
 u'Kunkka': 2,
 u'Legion Commander': 2,
 u'Leshrac': 3,
 u'Lich': 2,
 u'Lifestealer': 2,
 u'Lina': 3,
 u'Lion': 3,
 u'Lone Druid': 1,
 u'Luna': 1,
 u'Lycan': 2,
 u'Magnus': 2,
 u'Medusa': 1,
 u'Meepo': 1,
 u'Mirana': 1,


 u"Nature's Prophet": 3,
 u'Necrophos': 3,
 u'Night Stalker': 2,
 u'Nyx Assassin': 1,
 u'Ogre Magi': 2,
 u'Omniknight': 2,
 u'Oracle': 3,
 u'Outworld Devourer': 3,
 u'Phantom Assassin': 1,
 u'Phantom Lancer': 1,
 u'Phoenix': 2,
 u'Puck': 3,
 u'Pudge': 2,
 u'Pugna': 3,
 u'Queen of Pain': 3,
 u'Razor': 1,
 u'Riki': 1,
 u'Rubick': 3,
 u'Sand King': 2,
 u'Shadow Demon': 3,
 u'Shadow Fiend': 1,
 u'Shadow Shaman': 3,
 u'Silencer': 3,
 u'Skywrath Mage': 3,
 u'Slardar': 2,
 u'Slark': 1,
 u'Sniper': 1,
 u'Spectre': 1,
 u'Spirit Breaker': 2,
 u'Storm Spirit': 1,
 u'Sven': 1,
 u'Techies': 3,
 u'Templar Assassin': 1,
 u'Terrorblade': 1,
 u'Tidehunter': 2,
 u'Timbersaw': 2,
 u'Tinker': 3,
 u'Tiny': 2,
 u'Treant Protector': 2,
 u'Troll Warlord': 1,
 u'Tusk': 1,
 u'Undying': 3,
 u'Ursa': 2,
 u'Vengeful Spirit': 1,
 u'Venomancer': 1,
 u'Viper': 1,
 u'Visage': 3,
 u'Warlock': 3,
 u'Weaver': 2,
 u'Windrunner': 3,
 u'Winter Wyvern': 3,
 u'Witch Doctor': 3,
 u'Wraith King': 2,
 u'Zeus': 3}
{1: {u'BaseAgi'

 u'Night Stalker': 2,
 u'Nyx Assassin': 1,
 u'Ogre Magi': 2,
 u'Omniknight': 2,
 u'Oracle': 3,
 u'Outworld Devourer': 3,
 u'Phantom Assassin': 1,
 u'Phantom Lancer': 1,
 u'Phoenix': 2,
 u'Puck': 3,
 u'Pudge': 2,
 u'Pugna': 3,
 u'Queen of Pain': 3,
 u'Razor': 1,
 u'Riki': 1,
 u'Rubick': 3,
 u'Sand King': 2,
 u'Shadow Demon': 3,
 u'Shadow Fiend': 1,
 u'Shadow Shaman': 3,
 u'Silencer': 3,
 u'Skywrath Mage': 3,
 u'Slardar': 2,
 u'Slark': 1,
 u'Sniper': 1,
 u'Spectre': 1,
 u'Spirit Breaker': 2,
 u'Storm Spirit': 1,
 u'Sven': 1,
 u'Techies': 3,
 u'Templar Assassin': 1,
 u'Terrorblade': 1,
 u'Tidehunter': 2,
 u'Timbersaw': 2,
 u'Tinker': 3,
 u'Tiny': 2,
 u'Treant Protector': 2,
 u'Troll Warlord': 1,
 u'Tusk': 1,
 u'Undying': 3,
 u'Ursa': 2,
 u'Vengeful Spirit': 1,
 u'Venomancer': 1,
 u'Viper': 1,
 u'Visage': 3,
 u'Warlock': 3,
 u'Weaver': 2,
 u'Windrunner': 3,
 u'Winter Wyvern': 3,
 u'Witch Doctor': 3,
 u'Wraith King': 2,
 u'Zeus': 3}
{1: {u'BaseAgi': 22.41025641025641,
     u'BaseInt': 17.05

     u'BaseInt': 24.157894736842106,
     u'BaseStr': 18.842105263157894}}
(39, 34, 38)
33 32 28
not breaking
{u'Abaddon': 2,
 u'Alchemist': 3,
 u'Ancient Apparition': 3,
 u'Anti-Mage': 1,
 u'Arc Warden': 3,
 u'Axe': 2,
 u'Bane': 1,
 u'Batrider': 3,
 u'Beastmaster': 2,
 u'Bloodseeker': 1,
 u'Bounty Hunter': 1,
 u'Brewmaster': 1,
 u'Bristleback': 2,
 u'Broodmother': 1,
 u'Centaur Warrunner': 2,
 u'Chaos Knight': 2,
 u'Chen': 2,
 u'Clinkz': 1,
 u'Clockwerk': 2,
 u'Crystal Maiden': 2,
 u'Dark Seer': 3,
 u'Dazzle': 3,
 u'Death Prophet': 3,
 u'Disruptor': 3,
 u'Doom': 2,
 u'Dragon Knight': 1,
 u'Drow Ranger': 1,
 u'Earth Spirit': 2,
 u'Earthshaker': 2,
 u'Elder Titan': 3,
 u'Ember Spirit': 1,
 u'Enchantress': 1,
 u'Enigma': 3,
 u'Faceless Void': 1,
 u'Gyrocopter': 1,
 u'Huskar': 2,
 u'Invoker': 3,
 u'Io': 3,
 u'Jakiro': 3,
 u'Juggernaut': 1,
 u'Keeper of the Light': 3,
 u'Kunkka': 2,
 u'Legion Commander': 2,
 u'Leshrac': 3,
 u'Lich': 2,
 u'Lifestealer': 2,
 u'Lina': 3,
 u'Lion': 3,
 u'Lone 

 u'Elder Titan': 3,
 u'Ember Spirit': 1,
 u'Enchantress': 1,
 u'Enigma': 3,
 u'Faceless Void': 1,
 u'Gyrocopter': 1,
 u'Huskar': 2,
 u'Invoker': 3,
 u'Io': 3,
 u'Jakiro': 3,
 u'Juggernaut': 1,
 u'Keeper of the Light': 3,
 u'Kunkka': 2,
 u'Legion Commander': 2,
 u'Leshrac': 3,
 u'Lich': 2,
 u'Lifestealer': 2,
 u'Lina': 3,
 u'Lion': 3,
 u'Lone Druid': 1,
 u'Luna': 1,
 u'Lycan': 2,
 u'Magnus': 2,
 u'Medusa': 1,
 u'Meepo': 1,
 u'Mirana': 1,
 u'Morphling': 1,
 u'Naga Siren': 1,
 u"Nature's Prophet": 3,
 u'Necrophos': 3,
 u'Night Stalker': 2,
 u'Nyx Assassin': 1,
 u'Ogre Magi': 2,
 u'Omniknight': 2,
 u'Oracle': 3,
 u'Outworld Devourer': 3,
 u'Phantom Assassin': 1,
 u'Phantom Lancer': 1,
 u'Phoenix': 2,
 u'Puck': 3,
 u'Pudge': 2,
 u'Pugna': 3,
 u'Queen of Pain': 3,
 u'Razor': 1,
 u'Riki': 1,
 u'Rubick': 3,
 u'Sand King': 2,
 u'Shadow Demon': 3,
 u'Shadow Fiend': 1,
 u'Shadow Shaman': 3,
 u'Silencer': 3,
 u'Skywrath Mage': 3,
 u'Slardar': 2,
 u'Slark': 1,
 u'Sniper': 1,
 u'Spectre': 1,
 u'Spir

...

 u'Lycan': 2,
 u'Magnus': 2,
 u'Medusa': 1,
 u'Meepo': 1,
 u'Mirana': 1,
 u'Morphling': 1,
 u'Naga Siren': 1,
 u"Nature's Prophet": 3,
 u'Necrophos': 3,
 u'Night Stalker': 2,
 u'Nyx Assassin': 1,
 u'Ogre Magi': 2,
 u'Omniknight': 2,
 u'Oracle': 3,
 u'Outworld Devourer': 3,
 u'Phantom Assassin': 1,
 u'Phantom Lancer': 1,
 u'Phoenix': 2,
 u'Puck': 3,
 u'Pudge': 2,
 u'Pugna': 3,
 u'Queen of Pain': 3,
 u'Razor': 1,
 u'Riki': 1,
 u'Rubick': 3,
 u'Sand King': 2,
 u'Shadow Demon': 3,
 u'Shadow Fiend': 1,
 u'Shadow Shaman': 3,
 u'Silencer': 3,
 u'Skywrath Mage': 3,
 u'Slardar': 2,
 u'Slark': 1,
 u'Sniper': 1,
 u'Spectre': 1,
 u'Spirit Breaker': 2,
 u'Storm Spirit': 1,
 u'Sven': 1,
 u'Techies': 3,
 u'Templar Assassin': 1,
 u'Terrorblade': 1,
 u'Tidehunter': 2,
 u'Timbersaw': 2,
 u'Tinker': 3,
 u'Tiny': 2,
 u'Treant Protector': 2,
 u'Troll Warlord': 1,
 u'Tusk': 1,
 u'Undying': 3,
 u'Ursa': 2,
 u'Vengeful Spirit': 1,
 u'Venomancer': 1,
 u'Viper': 1,
 u'Visage': 3,
 u'Warlock': 3,
 u'Weaver': 2,


 u'Necrophos': 3,
 u'Night Stalker': 2,
 u'Nyx Assassin': 1,
 u'Ogre Magi': 2,
 u'Omniknight': 2,
 u'Oracle': 3,
 u'Outworld Devourer': 3,
 u'Phantom Assassin': 1,
 u'Phantom Lancer': 1,
 u'Phoenix': 2,
 u'Puck': 3,
 u'Pudge': 2,
 u'Pugna': 3,
 u'Queen of Pain': 3,
 u'Razor': 1,
 u'Riki': 1,
 u'Rubick': 3,
 u'Sand King': 2,
 u'Shadow Demon': 3,
 u'Shadow Fiend': 1,
 u'Shadow Shaman': 3,
 u'Silencer': 3,
 u'Skywrath Mage': 3,
 u'Slardar': 2,
 u'Slark': 1,
 u'Sniper': 1,
 u'Spectre': 1,
 u'Spirit Breaker': 2,
 u'Storm Spirit': 1,
 u'Sven': 1,
 u'Techies': 3,
 u'Templar Assassin': 1,
 u'Terrorblade': 1,
 u'Tidehunter': 2,
 u'Timbersaw': 2,
 u'Tinker': 3,
 u'Tiny': 2,
 u'Treant Protector': 2,
 u'Troll Warlord': 1,
 u'Tusk': 1,
 u'Undying': 3,
 u'Ursa': 2,
 u'Vengeful Spirit': 1,
 u'Venomancer': 1,
 u'Viper': 1,
 u'Visage': 3,
 u'Warlock': 3,
 u'Weaver': 2,
 u'Windrunner': 3,
 u'Winter Wyvern': 3,
 u'Witch Doctor': 3,
 u'Wraith King': 2,
 u'Zeus': 3}
{1: {u'BaseAgi': 22.41025641025641,
    

 u'Queen of Pain': 3,
 u'Razor': 1,
 u'Riki': 1,
 u'Rubick': 3,
 u'Sand King': 2,
 u'Shadow Demon': 3,
 u'Shadow Fiend': 1,
 u'Shadow Shaman': 3,
 u'Silencer': 3,
 u'Skywrath Mage': 3,
 u'Slardar': 2,
 u'Slark': 1,
 u'Sniper': 1,
 u'Spectre': 1,
 u'Spirit Breaker': 2,
 u'Storm Spirit': 1,
 u'Sven': 1,
 u'Techies': 3,
 u'Templar Assassin': 1,
 u'Terrorblade': 1,
 u'Tidehunter': 2,
 u'Timbersaw': 2,
 u'Tinker': 3,
 u'Tiny': 2,
 u'Treant Protector': 2,
 u'Troll Warlord': 1,
 u'Tusk': 1,
 u'Undying': 3,
 u'Ursa': 2,
 u'Vengeful Spirit': 1,
 u'Venomancer': 1,
 u'Viper': 1,
 u'Visage': 3,
 u'Warlock': 3,
 u'Weaver': 2,
 u'Windrunner': 3,
 u'Winter Wyvern': 3,
 u'Witch Doctor': 3,
 u'Wraith King': 2,
 u'Zeus': 3}
{1: {u'BaseAgi': 22.41025641025641,
     u'BaseInt': 17.05128205128205,
     u'BaseStr': 18.794871794871796},
 2: {u'BaseAgi': 15.5,
     u'BaseInt': 16.852941176470587,
     u'BaseStr': 22.323529411764707},
 3: {u'BaseAgi': 15.368421052631579,
     u'BaseInt': 24.157894736842106,
  

 u'Clinkz': 1,
 u'Clockwerk': 2,
 u'Crystal Maiden': 2,
 u'Dark Seer': 3,
 u'Dazzle': 3,
 u'Death Prophet': 3,
 u'Disruptor': 3,
 u'Doom': 2,
 u'Dragon Knight': 1,
 u'Drow Ranger': 1,
 u'Earth Spirit': 2,
 u'Earthshaker': 2,
 u'Elder Titan': 3,
 u'Ember Spirit': 1,
 u'Enchantress': 1,
 u'Enigma': 3,
 u'Faceless Void': 1,
 u'Gyrocopter': 1,
 u'Huskar': 2,
 u'Invoker': 3,
 u'Io': 3,
 u'Jakiro': 3,
 u'Juggernaut': 1,
 u'Keeper of the Light': 3,
 u'Kunkka': 2,
 u'Legion Commander': 2,
 u'Leshrac': 3,
 u'Lich': 2,
 u'Lifestealer': 2,
 u'Lina': 3,
 u'Lion': 3,
 u'Lone Druid': 1,
 u'Luna': 1,
 u'Lycan': 2,
 u'Magnus': 2,
 u'Medusa': 1,
 u'Meepo': 1,
 u'Mirana': 1,
 u'Morphling': 1,
 u'Naga Siren': 1,
 u"Nature's Prophet": 3,
 u'Necrophos': 3,
 u'Night Stalker': 2,
 u'Nyx Assassin': 1,
 u'Ogre Magi': 2,
 u'Omniknight': 2,
 u'Oracle': 3,
 u'Outworld Devourer': 3,
 u'Phantom Assassin': 1,
 u'Phantom Lancer': 1,
 u'Phoenix': 2,
 u'Puck': 3,
 u'Pudge': 2,
 u'Pugna': 3,
 u'Queen of Pain': 3,
 u'Raz

 u'Io': 3,
 u'Jakiro': 3,
 u'Juggernaut': 1,
 u'Keeper of the Light': 3,
 u'Kunkka': 2,
 u'Legion Commander': 2,
 u'Leshrac': 3,
 u'Lich': 2,
 u'Lifestealer': 2,
 u'Lina': 3,
 u'Lion': 3,
 u'Lone Druid': 1,
 u'Luna': 1,
 u'Lycan': 2,
 u'Magnus': 2,
 u'Medusa': 1,
 u'Meepo': 1,
 u'Mirana': 1,
 u'Morphling': 1,
 u'Naga Siren': 1,
 u"Nature's Prophet": 3,
 u'Necrophos': 3,
 u'Night Stalker': 2,
 u'Nyx Assassin': 1,
 u'Ogre Magi': 2,
 u'Omniknight': 2,
 u'Oracle': 3,
 u'Outworld Devourer': 3,
 u'Phantom Assassin': 1,
 u'Phantom Lancer': 1,
 u'Phoenix': 2,
 u'Puck': 3,
 u'Pudge': 2,
 u'Pugna': 3,
 u'Queen of Pain': 3,
 u'Razor': 1,
 u'Riki': 1,
 u'Rubick': 3,
 u'Sand King': 2,
 u'Shadow Demon': 3,
 u'Shadow Fiend': 1,
 u'Shadow Shaman': 3,
 u'Silencer': 3,
 u'Skywrath Mage': 3,
 u'Slardar': 2,
 u'Slark': 1,
 u'Sniper': 1,
 u'Spectre': 1,
 u'Spirit Breaker': 2,
 u'Storm Spirit': 1,
 u'Sven': 1,
 u'Techies': 3,
 u'Templar Assassin': 1,
 u'Terrorblade': 1,
 u'Tidehunter': 2,
 u'Timbersaw': 2,
